# Support Vector Machine (SVM) Implementation on Handwritten Digits Dataset

This notebook demonstrates a self-driven analysis using Support Vector Machines (SVM) for classifying handwritten digits. The workflow includes data preprocessing, visualization, model training with different kernels, hyperparameter tuning, and robustness evaluation.

### Import Required Libraries

- **Import essential libraries** for data manipulation, visualization, and machine learning
- These imports are foundational for all subsequent analysis

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
import seaborn as sns

### Load and Prepare Dataset

- **Load the digits dataset** from scikit-learn
- **Extract features and labels** for modeling
- This step sets up the data for further analysis

In [2]:
digits = load_digits()
X = digits.data
y = digits.target

### Data Preparation and Adding Noise

- **Split the dataset** into training and test sets
- **Add Gaussian noise** to 50% of the training data to test model robustness
- Prepares data for evaluating SVM performance under noisy conditions

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

# Add noise to 50% of training data
num_noisy = int(0.5 * len(X_train))
indices = np.random.choice(len(X_train), num_noisy, replace=False)

# Gaussian noise
noise = np.random.normal(0, 8, size=X_train[indices].shape)
X_train_noisy = X_train.copy()
X_train_noisy[indices] += noise

images = digits.images[y_train]

### Visualize Sample Digits

- **Display sample digit images** from the dataset
- Provides visual context for the classification task

In [ ]:
plt.figure(figsize=(10, 2))
for i in range(10):
    plt.subplot(1, 10, i + 1)
    plt.imshow(images[i], cmap='gray')
    plt.title(f'Label: {y[i]}')
    plt.axis('off')
plt.suptitle("Sample Digits (0 to 4)", fontsize=16)
plt.show()

### Check Data Shapes

- **Display the shapes** of training and test sets
- Confirms the data split and readiness for modeling

In [ ]:
X_test.shape, X_train.shape

### PCA for Visualization

- **Reduce data dimensionality** to 2 components using PCA
- Facilitates visualization of class separation in 2D space

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_train)

### Visualize PCA Projection

- **Plot the 2D PCA projection** of the training data
- Visualizes how well digit classes are separated after dimensionality reduction

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=y_train, palette='tab10', alpha=0.7, legend='full')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('PCA Projection of Digits 0-4')
plt.legend(title="Digit Label", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

### Train SVM with Linear Kernel

- **Train a Support Vector Machine** with a linear kernel on noisy training data
- **Evaluate accuracy** on the test set
- Establishes a baseline for SVM performance

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm_linear = SVC(kernel='linear', C=1.0)
svm_linear.fit(X_train_noisy, y_train)
y_pred_linear = svm_linear.predict(X_test)

print("Linear SVM Accuracy:", accuracy_score(y_test, y_pred_linear))

### Standardize Features

- **Standardize features** to zero mean and unit variance
- Prepares data for SVMs with RBF or polynomial kernels
- Improves model performance and stability

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_noisy)
X_test_scaled = scaler.transform(X_test)

### Train SVM with RBF Kernel

- **Train an SVM** with an RBF (Radial Basis Function) kernel on standardized data
- **Evaluate accuracy** on the test set
- Compares performance with linear kernel

In [ ]:
svm_rbf = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_rbf.fit(X_train_scaled, y_train)
y_pred_rbf = svm_rbf.predict(X_test_scaled)

print("RBF SVM Accuracy:", accuracy_score(y_test, y_pred_rbf))

### Hyperparameter Tuning with GridSearchCV

- **Perform grid search** to find optimal SVM hyperparameters (kernel, C, gamma, degree)
- **Train and evaluate** models using cross-validation
- Identifies the best configuration for improved accuracy

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'kernel': ['rbf', 'poly'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 0.001, 0.01, 0.1],
    'degree': [1,2,3,4, 5, 6]
}

grid = GridSearchCV(SVC(), param_grid=param_grid, cv=10)
grid.fit(X_train_scaled, y_train)
print("Best Parameters:", grid.best_params_)

### Train SVM with Best Parameters

- **Train an SVM** with the best parameters found from grid search
- **Evaluate performance** on the test set
- Demonstrates the impact of hyperparameter tuning

In [ ]:
svc = SVC(C=1, degree=3, gamma=0.1, kernel='poly')
svc.fit(X_train_scaled, y_train)
print("RBF SVM Accuracy:", accuracy_score(y_test, y_pred_rbf))

### Try Robust SVM Variants (NuSVC)

- **Experiment with NuSVC**, a robust SVM variant
- **Tune hyperparameters** using grid search
- Compares alternative SVM formulations for potential performance gains

In [ ]:
from sklearn.svm import NuSVC
param_grid = { 'nu': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7],
    'kernel': ['rbf', 'poly'],
    'gamma': ['scale', 0.001, 0.01, 0.1],
    'degree': [1,2,3,4, 5, 6]
}

grid =  GridSearchCV(NuSVC(random_state=42), param_grid=param_grid, cv=5, n_jobs=-1)
grid.fit(X_train_scaled, y_train)
print("Best Parameters:", grid.best_params_)

### Evaluate Best NuSVC Model

- **Train and evaluate** the best NuSVC model found from grid search
- **Report accuracy** on the test set
- Assesses the effectiveness of robust SVM variants

In [ ]:
nusvc_best = grid.best_estimator_
nusvc_best.fit(X_train_scaled, y_train)
y_pred_nusvc = nusvc_best.predict(X_test_scaled)
print("NuSVC Best Accuracy:", accuracy_score(y_test, y_pred_nusvc))

### Conclusion

- **Summarize findings** from SVM experiments
- Highlight the impact of noise, feature scaling, and hyperparameter tuning
- Emphasize the importance of data quality for model performance

## Final Summary

**Objective:**
- Classify handwritten digits using Support Vector Machines (SVM) and analyze the impact of noise, feature scaling, and model tuning.

**Steps Performed:**
- Imported and explored the scikit-learn digits dataset.
- Added Gaussian noise to training data to test model robustness.
- Visualized data using PCA and sample digit images.
- Trained SVM models with linear, RBF, and polynomial kernels.
- Applied feature standardization and hyperparameter tuning with GridSearchCV.
- Explored robust SVM variants (NuSVC) and compared their performance.

**Key Results:**
- Feature scaling and kernel choice significantly affected SVM performance.
- Hyperparameter tuning improved accuracy, but noise limited maximum achievable performance.
- NuSVC and other variants did not outperform standard SVM under noisy conditions.

**Next Steps:**
- Investigate advanced noise reduction or feature engineering techniques.
- Explore ensemble methods or deep learning for further accuracy improvements.
- Consider cross-validation for more robust model evaluation.